Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST_sanitized.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (200000, 28, 28), (200000,))
('Validation set', (9633, 28, 28), (9633,))
('Test set', (9604, 28, 28), (9604,))


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (200000, 784), (200000, 10))
('Validation set', (9633, 784), (9633, 10))
('Test set', (9604, 784), (9604, 10))


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) 
            / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compue the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
# From previous problem set, with modifications

batch_size = 128
num_relus = 1024
l2_w1par = 0.01
l2_w2par = 0.01


graph = tf.Graph()
with graph.as_default():

    # For training data, use a placeholder that will be fed a minibatch at runtime.
    tf_train_dataset = tf.placeholder(tf.float32, 
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    # our network now has one set of weight and biases producing an output vector of
    # size `num_relus`. we apply the relus and then feed that vector into a new set 
    # of weight matrix + bias vector that will produce a vector of sime `num_labels`
    # that is fed into our softmax
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_relus]))
    biases1 = tf.Variable(tf.zeros([num_relus]))
    weights2 = tf.Variable(tf.truncated_normal([num_relus, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))

  
    # Training computation.
    logits1 = tf.matmul(tf_train_dataset, weights1) + biases1
    activations = tf.nn.relu(logits1)
    logits2 = tf.matmul(activations, weights2) + biases2

    # add L2 loss terms: https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#l2_loss
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits2, tf_train_labels) + 
                         l2_w1par * tf.nn.l2_loss(weights1) + 
                         l2_w2par * tf.nn.l2_loss(weights2))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits2)
    # validation set...
    val_logits1 = tf.matmul(tf_valid_dataset, weights1) + biases1
    val_activations = tf.nn.relu(val_logits1)
    val_logits2 = tf.matmul(val_activations, weights2) + biases2
    valid_prediction = tf.nn.softmax(val_logits2)
    # test set...
    test_logits1 = tf.matmul(tf_test_dataset, weights1) + biases1
    test_activations = tf.nn.relu(test_logits1)
    test_logits2 = tf.matmul(test_activations, weights2) + biases2
    test_prediction = tf.nn.softmax(test_logits2) 

In [6]:
# From previous problem set, with modifications

num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                    valid_prediction.eval(), valid_labels))
            
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3413.484375
Minibatch accuracy: 13.3%
Validation accuracy: 37.8%
Minibatch loss at step 500: 21.491398
Minibatch accuracy: 75.0%
Validation accuracy: 80.2%
Minibatch loss at step 1000: 0.911050
Minibatch accuracy: 86.7%
Validation accuracy: 79.4%
Minibatch loss at step 1500: 0.827016
Minibatch accuracy: 79.7%
Validation accuracy: 79.4%
Minibatch loss at step 2000: 0.677288
Minibatch accuracy: 87.5%
Validation accuracy: 79.9%
Minibatch loss at step 2500: 0.691102
Minibatch accuracy: 84.4%
Validation accuracy: 79.4%
Minibatch loss at step 3000: 0.635151
Minibatch accuracy: 88.3%
Validation accuracy: 79.7%
Test accuracy: 90.1%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [8]:
train_dataset = train_dataset[:(128*3)]
train_labels = train_labels[:(128*3)]

In [10]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                    valid_prediction.eval(), valid_labels))
            
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3545.514893
Minibatch accuracy: 10.9%
Validation accuracy: 21.6%
Minibatch loss at step 500: 21.068083
Minibatch accuracy: 100.0%
Validation accuracy: 68.7%
Minibatch loss at step 1000: 0.403465
Minibatch accuracy: 100.0%
Validation accuracy: 70.0%
Minibatch loss at step 1500: 0.243713
Minibatch accuracy: 100.0%
Validation accuracy: 70.3%
Minibatch loss at step 2000: 0.233255
Minibatch accuracy: 100.0%
Validation accuracy: 70.3%
Minibatch loss at step 2500: 0.228013
Minibatch accuracy: 100.0%
Validation accuracy: 70.5%
Minibatch loss at step 3000: 0.224631
Minibatch accuracy: 100.0%
Validation accuracy: 70.4%
Test accuracy: 81.4%


Here, the minibatch accuracy quickly saturates and as a consequence, the model stops evolving and performs poorly at the test stage.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
